In [81]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.svm import SVC
from sklearn.metrics import f1_score

In [82]:
df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')
df

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595207,1488013,0,3,1,10,0,0,0,0,0,...,4,1,9,6,0,1,1,0,1,1
595208,1488016,0,5,1,3,0,0,0,0,0,...,4,1,3,8,1,0,1,0,1,1
595209,1488017,0,1,1,10,0,0,1,0,0,...,3,2,2,6,0,0,1,0,0,0
595210,1488021,0,5,2,3,1,0,0,0,1,...,4,1,4,2,0,1,1,1,0,0


### 数值型特征

In [83]:
df.describe()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,5.952120e+05,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,...,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000
mean,7.438036e+05,0.036448,1.900378,1.358943,4.423318,0.416794,0.405188,0.393742,0.257033,0.163921,...,5.441382,1.441918,2.872288,7.539026,0.122427,0.627840,0.554182,0.287182,0.349024,0.153318
std,4.293678e+05,0.187401,1.983789,0.664594,2.699902,0.493311,1.350642,0.488579,0.436998,0.370205,...,2.332871,1.202963,1.694887,2.746652,0.327779,0.483381,0.497056,0.452447,0.476662,0.360295
min,7.000000e+00,0.000000,0.000000,-1.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.719915e+05,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.435475e+05,0.000000,1.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,1.000000,3.000000,7.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,1.115549e+06,0.000000,3.000000,2.000000,6.000000,1.000000,0.000000,1.000000,1.000000,0.000000,...,7.000000,2.000000,4.000000,9.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,1.488027e+06,1.000000,7.000000,4.000000,11.000000,1.000000,6.000000,1.000000,1.000000,1.000000,...,19.000000,10.000000,13.000000,23.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### 查看特征缺失比例

In [84]:
for column in df.columns:
    if column in ['id', 'target']:
        continue
    print('%s missing value rate: %s' % (column, (df[column]==-1).sum()/len(df)))

ps_ind_01 missing value rate: 0.0
ps_ind_02_cat missing value rate: 0.0003628959093566662
ps_ind_03 missing value rate: 0.0
ps_ind_04_cat missing value rate: 0.00013944611331760784
ps_ind_05_cat missing value rate: 0.00975954785857812
ps_ind_06_bin missing value rate: 0.0
ps_ind_07_bin missing value rate: 0.0
ps_ind_08_bin missing value rate: 0.0
ps_ind_09_bin missing value rate: 0.0
ps_ind_10_bin missing value rate: 0.0
ps_ind_11_bin missing value rate: 0.0
ps_ind_12_bin missing value rate: 0.0
ps_ind_13_bin missing value rate: 0.0
ps_ind_14 missing value rate: 0.0
ps_ind_15 missing value rate: 0.0
ps_ind_16_bin missing value rate: 0.0
ps_ind_17_bin missing value rate: 0.0
ps_ind_18_bin missing value rate: 0.0
ps_reg_01 missing value rate: 0.0
ps_reg_02 missing value rate: 0.0
ps_reg_03 missing value rate: 0.18106489788512328
ps_car_01_cat missing value rate: 0.0001797678810239041
ps_car_02_cat missing value rate: 8.400368272145051e-06
ps_car_03_cat missing value rate: 0.6908983689844

* 缺失比例较大的feature: ps_reg_03, ps_car_03_cat, ps_car_05_cat, ps_car_14

### 类别型特征

In [85]:
category_columns = [column for column in df.columns if 'cat' in column]
for column in category_columns:
    category_proportion = df[column].value_counts(normalize=True)
    print(category_proportion)

ps_ind_02_cat
 1    0.725555
 2    0.207612
 3    0.047355
 4    0.019116
-1    0.000363
Name: proportion, dtype: float64
ps_ind_04_cat
 0    0.582927
 1    0.416934
-1    0.000139
Name: proportion, dtype: float64
ps_ind_05_cat
 0    0.887094
 6    0.034714
 4    0.030819
 1    0.013982
 3    0.013832
-1    0.009760
 2    0.007029
 5    0.002770
Name: proportion, dtype: float64
ps_car_01_cat
 11    0.348738
 7     0.301148
 6     0.104825
 10    0.084150
 4     0.043974
 9     0.034144
 5     0.030480
 8     0.025357
 3     0.011186
 0     0.009919
 2     0.003602
 1     0.002297
-1     0.000180
Name: proportion, dtype: float64
ps_car_02_cat
 1    0.829940
 0    0.170052
-1    0.000008
Name: proportion, dtype: float64
ps_car_03_cat
-1    0.690898
 1    0.185999
 0    0.123102
Name: proportion, dtype: float64
ps_car_04_cat
0    0.834293
1    0.053956
2    0.039935
8    0.034606
9    0.031979
6    0.002621
3    0.001075
5    0.000916
4    0.000386
7    0.000234
Name: proportion, dtype: f

In [86]:
## 将类别特征处理成哑变量
dummies = pd.get_dummies(df[category_columns].astype(str))
test_dummies = pd.get_dummies(test_df[category_columns].astype(str))

In [87]:
df = pd.concat([df, dummies], axis=1)
test_df = pd.concat([test_df, test_dummies], axis=1)

In [88]:
print(len(df.columns))
print(len(test_df.columns))

243
242


In [92]:
df[df.columns.drop(category_columns)].to_csv('../data/train_dumpies.csv', index=False)
test_df[test_df.columns.drop(category_columns)].to_csv('../data/test_dumpies.csv', index=False)